In [3]:
import pysr
from pysr import PySRRegressor # Main tool for regression
import numpy as np
import sympy # Represent Equations

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


In [38]:
# Testing PySR using the 3D Distance Formula

rng = np.random.default_rng()

points = {
    'x1': rng.uniform(0, 10, size=1000).reshape(-1, 1),
    'x2': rng.uniform(0, 10, size=1000).reshape(-1, 1),
    'y1': rng.uniform(0, 10, size=1000).reshape(-1, 1),
    'y2': rng.uniform(0, 10, size=1000).reshape(-1, 1),
    'z1': rng.uniform(0, 10, size=1000).reshape(-1, 1),
    'z2': rng.uniform(0, 10, size=1000).reshape(-1, 1),
}

distances = np.sqrt(((points['x2']-points['x1'])**2)+((points['y2']-points['y1'])**2)+((points['z2']-points['z1'])**2))

In [39]:
inputs = np.concatenate(list(points.values()), axis = 1)
print(inputs.shape)

(1000, 6)


In [77]:
# Base Model

model = PySRRegressor(
    maxsize=20,
    niterations=100,
    populations=31,
    population_size = 27,
    ncycles_per_iteration = 760,
    binary_operators=["+"],
    unary_operators = ["square", "sqrt"],
    temp_equation_file = True,
    tempdir = 'outputs',
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

In [78]:
model.fit(inputs, distances, variable_names = list(points.keys()))

/home/alang/anaconda3/envs/ParT/lib/python3.10/site-packages/pysr/sr.py:2811: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!



Expressions evaluated per second: 4.440e+05
Progress: 1379 / 6200 total iterations (22.242%)
════════════════════════════════════════════════════════════════════════════════════════════════════
───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           6.233e+00  0.000e+00  y = 6.604
5           6.184e+00  1.970e-03  y = (z2 * -0.076433) + 6.9776
6           5.118e+00  1.892e-01  y = abs(x2 - x1) + 3.2128
7           4.822e+00  5.948e-02  y = sqrt(abs(x2 - x1)) + 4.9026
8           4.178e+00  1.434e-01  y = abs((x2 - x1) * -0.59656) + 4.5808
10          4.136e+00  5.082e-03  y = sqrt(abs(square(x2 - x1)) + 21.667) + 0.55328
14          2.531e+00  1.228e-01  y = sqrt(abs((y2 - y1) * -7.0487) + abs((x1 - x2) / -0.139...
                                      16))
16          2.518e+00  2.463e-03  y = sqrt(abs((y2 - y1) * -7.0487) + abs(((x1 - x2) + 0.285...
                                    

[ Info: Final population:
[ Info: Results saved to:


PySRRegressor.equations_ = [
	    pick      score                                           equation  \
	0          0.000000                                          6.6040177   
	1          0.002460                               sqrt(48.558327 - z2)   
	2          0.005530                      (0.17129982 / x1) + 6.5048103   
	3          0.184177                           abs(x1 - x2) + 3.2130637   
	4          0.211457                  sqrt(square(x1 - x2) + 28.038177)   
	5          0.013122                        abs(z1 - z2) + abs(x2 - x1)   
	6          0.066309         (abs(x1 - x2) + abs(z1 - z2)) * 0.90577304   
	7          0.515247  sqrt((square(x1 - x2) + square(z1 - z2)) + 13....   
	8          0.003582  sqrt(square(z1 - z2) + (square(x1 - x2) + 8.70...   
	9   >>>>  30.214445  sqrt((square(x2 - x1) + square(y1 - y2)) + squ...   
	10         0.009357  sqrt(square(z2 - z1) + ((1.6625427e-7 + square...   
	11         0.003419  sqrt(square(z1 - z2) + ((square(x1 + (1.628227...   
	
	            loss  complexity  
	0   6.232854e+00           1  
	1   6.187017e+00           4  
	2   6.152894e+00           5  
	3   5.117909e+00           6  
	4   4.142455e+00           7  
	5   4.035155e+00           9  
	6   3.533985e+00          11  
	7   2.111036e+00          12  
	8   2.095967e+00          14  
	9   1.582769e-13          15  
	10  1.553424e-13          17  
	11  1.542837e-13          19  
]

  - /tmp/tmphtomg1lz/20251127_204815_WY7LYr/hall_of_fame.csv


In [76]:
print(model.sympy())

sqrt((x1 - x2)**2 + (-y1 + y2)**2 + (-z1 + z2)**2)


In [ ]:
# What if we add some operators not part of the original equation?

model = PySRRegressor(
    maxsize=20,
    niterations=100,
    populations=31,
    population_size = 27,
    ncycles_per_iteration = 760,
    binary_operators=["+", "-", "*", "/"],
    unary_operators = ["square", "sqrt", "abs", "cos", "tan"],
    temp_equation_file = True,
    tempdir = 'outputs',
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

model.fit(inputs, distances, variable_names = list(points.keys()))

print(model.sympy())

In [ ]:
# The model has a much lower chance to find the equation due to the increased search space. Let's counteract this by tripling niterations

model = PySRRegressor(
    maxsize=20,
    niterations=300,
    populations=31,
    population_size = 27,
    ncycles_per_iteration = 760,
    binary_operators=["+", "-", "*", "/"],
    unary_operators = ["square", "sqrt", "abs", "cos", "tan"],
    temp_equation_file = True,
    tempdir = 'outputs',
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

model.fit(inputs, distances, variable_names = list(points.keys()))

print(model.sympy())

In [ ]:
# Increasing iterations increases local search quality. What about number of populations?

model = PySRRegressor(
    maxsize=20,
    niterations=100,
    populations=93,
    population_size = 27,
    ncycles_per_iteration = 760,
    binary_operators=["+", "-", "*", "/"],
    unary_operators = ["square", "sqrt", "abs", "cos", "tan"],
    temp_equation_file = True,
    tempdir = 'outputs',
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

model.fit(inputs, distances, variable_names = list(points.keys()))

print(model.sympy())

In [ ]:
# Increasing the number of populations increases global search opportunities. What about population size?

model = PySRRegressor(
    maxsize=20,
    niterations=100,
    populations=31,
    population_size = 81,
    ncycles_per_iteration = 760,
    binary_operators=["+", "-", "*", "/"],
    unary_operators = ["square", "sqrt", "abs", "cos", "tan"],
    temp_equation_file = True,
    tempdir = 'outputs',
    elementwise_loss="loss(prediction, target) = (prediction - target)^2")

model.fit(inputs, distances, variable_names = list(points.keys()))

print(model.sympy())